In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


Following the, Store Sales TS Forecasting - A Comprehensive Guide

In [2]:
#Basic imports
import numpy as np
import pandas as pd
import os
import gc
import warnings

In [3]:
#PACF - ACF
import statsmodels.api as sm

In [4]:
#Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [6]:
#config
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format #unclear, define this config
warnings.filterwarnings('ignore')

In [7]:
def readcsv(path):
    df = pd.read_csv(path)
    return df

In [8]:
train = readcsv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = readcsv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = readcsv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

#transactions will be sorted with '.sort_values["~", "date"]'
transactions = readcsv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

### In time series, it is important to set dtype of time into 'datetime'. It allows time manitulation.

In [9]:
#Datetime 'pd.to_datetime(~.date)'
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

In [10]:
# 'astype' is a method of changing dataframe's dtype.
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

In [11]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0.00


## Transactions